In [2]:
import sys
sys.path.append("../")
sys.path.append("../data_retrieval/data")
from data_retrieval.data import split_data
from torchvision.models import resnet101, ResNet101_Weights
import torch
import json
import torch
from sklearn.metrics import confusion_matrix

In [3]:
train, val, test = split_data.split_data('../data_retrieval/data/annotations.csv', '../data_retrieval/data/', ResNet101_Weights.IMAGENET1K_V2.transforms())

In [4]:
model = resnet101()

In [5]:
model.load_state_dict(torch.load('../models/models_weights/resnet101.pth'))

<All keys matched successfully>

In [6]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
test_loader = torch.utils.data.DataLoader(test,
                                             batch_size=16, shuffle=True,
                                             num_workers=4)

In [8]:
f = open("../data_retrieval/data/mapping")
mapping = json.load(f)

In [9]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in mapping.values()}
total_pred = {classname: 0 for classname in mapping.values()}

# again no gradients needed
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[mapping[str(int(label))]] += 1
            total_pred[mapping[str(int(label))]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: E_Neo-Assyrian is 100.0 %
Accuracy for class: E_Neo-Babylonian is 91.5 %
Accuracy for class: GAR_Neo-Assyrian is 89.7 %
Accuracy for class: GAR_Neo-Babylonian is 96.8 %
Accuracy for class: KA_Neo-Assyrian is 90.0 %
Accuracy for class: KA_Neo-Babylonian is 85.4 %
Accuracy for class: KI_Neo-Assyrian is 97.4 %
Accuracy for class: KI_Neo-Babylonian is 100.0 %
Accuracy for class: MEŠ_Neo-Assyrian is 100.0 %
Accuracy for class: MEŠ_Neo-Babylonian is 94.1 %
Accuracy for class: NI_Neo-Assyrian is 90.3 %
Accuracy for class: NI_Neo-Babylonian is 92.3 %
Accuracy for class: RU_Neo-Assyrian is 80.0 %
Accuracy for class: RU_Neo-Babylonian is 90.0 %
Accuracy for class: TA_Neo-Assyrian is 80.0 %
Accuracy for class: TA_Neo-Babylonian is 93.3 %
Accuracy for class: TI_Neo-Assyrian is 100.0 %
Accuracy for class: TI_Neo-Babylonian is 89.7 %
Accuracy for class: U₂_Neo-Assyrian is 73.1 %
Accuracy for class: U₂_Neo-Babylonian is 93.8 %
Accuracy for class: ŠU_Neo-Assyrian is 81.8 %
Accuracy

In [10]:
print("Overall accuracy: ",sum(correct_pred.values()) / sum(total_pred.values()))

Overall accuracy:  0.9209401709401709
